In [1]:
import collections

from owslib.wms import WebMapService
import matplotlib.pyplot as plt
import skimage.io, numpy
from tqdm import tqdm_notebook as tqdm
import skimage.color
import multiprocessing.pool

In [2]:
import xml.etree.ElementTree as ET
root = ET.parse('epilzn.xml').getroot()
rules = list(root[0][1][1])

In [3]:
colour_to_rule = {}
for rule in rules:
    name = rule[0].text
    colour = rule[3][0][0].text
    colour = int(colour[1:3], 16), int(colour[3:5], 16), int(colour[5:7], 16)
    colour_to_rule[colour] = name
colour_to_rule

{(201, 255, 249): 'B1',
 (98, 240, 245): 'B2',
 (0, 194, 237): 'B3',
 (149, 157, 194): 'B4',
 (125, 160, 171): 'B5',
 (149, 191, 204): 'B6',
 (186, 214, 222): 'B7',
 (200, 230, 224): 'B8',
 (255, 255, 255): 'UL',
 (230, 153, 0): 'E1',
 (240, 174, 60): 'E2',
 (247, 197, 104): 'E3',
 (255, 218, 150): 'E4',
 (221, 184, 245): 'IN1',
 (243, 219, 255): 'IN2',
 (197, 149, 232): 'IN3',
 (174, 115, 222): 'IN4',
 (255, 207, 255): 'R1',
 (255, 166, 163): 'R2',
 (255, 119, 110): 'R3',
 (255, 72, 59): 'R4',
 (255, 217, 217): 'R5',
 (85, 255, 0): 'RE1',
 (211, 255, 190): 'RE2',
 (237, 216, 173): 'RU1',
 (230, 202, 151): 'RU2',
 (222, 192, 131): 'RU3',
 (214, 188, 111): 'RU4',
 (214, 161, 156): 'RU5',
 (199, 158, 76): 'RU6',
 (255, 255, 161): 'SP1',
 (255, 255, 112): 'SP2',
 (255, 255, 0): 'SP3',
 (255, 127, 99): 'UD',
 (217, 255, 242): 'W1',
 (153, 255, 221): 'W2',
 (51, 255, 187): 'W3',
 (209, 95, 185): None}

In [34]:
import shapefile
shape = shapefile.Reader('shapesnsw/MB_2016_NSW')

sa2_to_name = {}
sa2_to_coords = {}
sa2_to_points = {}
sa2_seven_to_sixteen = {}
for sr in tqdm(shape.shapeRecords()):
    sa2, name = sr.record.SA1_MAIN16, sr.record.SA2_NAME16
    sa2_to_name[int(sa2)] = name
    sa2_seven_to_sixteen[sr.record.SA1_7DIG16] = sr.record.SA1_MAIN16
    if len(sr.shape.points) > 0:
        sa2_to_coords[sa2] = numpy.mean(sr.shape.points, axis=0)
        sa2_to_points[sa2] = sr.shape.points

In [35]:
with open('sa1_seven_to_sixteen.json', 'w') as f:
    json.dump(sa2_seven_to_sixteen, f)

In [16]:
sa2s = sorted([sa2 for sa2 in list(sa2_to_coords) if 116 <= int(sa2[:3]) <= 128])

In [17]:
sr.record.SA1_MAIN16

'19797979992'

In [18]:
wms = WebMapService('https://mapprod1.environment.nsw.gov.au/arcgis/services/Planning/EPI_Primary_Planning_Layers/MapServer/WMSServer?request=GetCapabilities&service=WMS', version='1.1.1')

In [19]:
list(wms.contents)

['0', '1', '2', '3', '4', '5', '6']

In [20]:
wms['4'].title

'Land Zoning'

In [21]:
wms['4'].styles

{'default': {'title': '4',
  'legend': 'http://mapprod1.environment.nsw.gov.au/arcgis/services/Planning/EPI_Primary_Planning_Layers/MapServer/WmsServer?request=GetLegendGraphic%26version=1.1.1%26format=image/png%26layer=4'}}

In [22]:
[op.name for op in wms.operations]

['GetCapabilities', 'GetMap', 'GetFeatureInfo', 'GetStyles']

In [23]:
c_zones = {'B1', 'B2', 'B3', 'B4', 'B5',
           'B6', 'B7', 'B8', 'SP3'}
park_zones = {'E1', 'E2', 'E3', 'E4', 'RE1',
              'RE2', 'RU1', 'RU2', 'RU3',
              'RU4'}
water_zones = {'W1', 'W2', 'W3'}
i_zones = {'IN1', 'IN2', 'IN3', 'IN4'}
r_zones = {'R1', 'R2', 'R3', 'R4', 'R5',
           'RU5'}

In [24]:
def get_zoning(sa2, timeout=10):
    points = sa2_to_points[sa2]
    bbox = numpy.concatenate([numpy.min(points, axis=0),
                              numpy.max(points, axis=0)])
    # Only need to do this if you're masking.
#     width = bbox[2] - bbox[0]
#     height = bbox[3] - bbox[1]
#     points = numpy.array(points)
#     points[:, 0] -= bbox[0]
#     points[:, 0] /= width / 100
#     points[:, 1] -= bbox[1]
#     points[:, 1] /= height / 100
    while True:
        try:
            img = wms.getmap(
                layers=['4'],
                styles=['default'],
                srs='EPSG:4326',
                bbox=tuple(bbox),
                size=(100, 100),
                format='image/png',
                timeout=timeout)
            print('Got image for', sa2)
            break
        except KeyboardInterrupt:
            break
        except:
            print('Connect timeout on', sa2)
            continue
    with open(f'itsajpg-{sa2}.jpg', 'wb') as out:
        out.write(img.read())
    img = skimage.io.imread(f'itsajpg-{sa2}.jpg')
    zones = []
    for px in img.reshape(-1, 3):
        px = tuple(px)
        if px == (0, 0, 0):
            continue

        rule = colour_to_rule.get(px, 'U')
        if rule in c_zones:
            z = 'C'
        elif rule in park_zones:
            z = 'P'
        elif rule in water_zones:
            z = 'W'
        elif rule in i_zones:
            z = 'I'
        elif rule in r_zones:
            z = 'R'
        else:
            z = 'U'
        zones.append(z)
    return collections.Counter(zones)

In [ ]:
# sa2_to_zoning = {}
pool = multiprocessing.pool.Pool(12)
zones = list(tqdm(pool.imap(get_zoning, sa2s), total=len(sa2s)))

In [ ]:
# for sa2 in tqdm(sa2s):
#     sa2_to_zoning[sa2] = get_zoning(sa2)

In [ ]:
get_zoning(sa2, timeout=30)

In [27]:
len(zones)

9818

In [28]:
import json

In [31]:
with open('zone-info-sydney.json', 'w') as f:
    json.dump(dict(zip(sa2s, zones)), f)